# Autogenサンプル-グループチャット（メニュー考案）

Autogenサンプル-グループチャット（メニュー考案） - Colaboratory https://colab.research.google.com/drive/1r4TtIxOCktNgTdAR9f4rVgV6iXiR0wa_?usp=sharing#scrollTo=hrtTi-dUIUyL

## Autogenについて
Microsoft Researchから登場したマルチエージェントによるLLMアプリケーションを実現するフレームワーク。役割を設定したエージェントを定義することで、エージェントがお互いに相談しながらタスクを実行できる。インターネットアクセスやコードの実行、ファンクションなども活用しながら高度な操作ができる。

公式サイト
https://microsoft.github.io/autogen/

Github
https://github.com/microsoft/autogen

ブログ
AutoGen: Enabling next-generation large language model applications
https://www.microsoft.com/en-us/research/blog/autogen-enabling-next-generation-large-language-model-applications/

arxivペーパー
AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
https://arxiv.org/abs/2308.08155


## 注意

- 実行には、OpenAIのAPIキーが必要。
- 複雑な処理や高度な連携にはGPT-4のアクセスが必要。
- コードの修正などが繰り返されるケースではAPIアクセスが多発しコストが嵩むので注意。
- タスクによっては、エージェント間でコードやソースなどを共有する必要があり、大量の入力が発生するため、16Kや32Kに対応したモデルでないと動作しない（Rate Limitエラーにひっかかる）。
- Web情報の取得やデータ分析などは16K、32Kモデル必須。

In [15]:
AOAIKey = input("Enter your API key: ")
AOAIEndpoint = input("Enter your endpoint: ")

In [19]:
import autogen

config_list = [
    # {
    #     "model": "gpt-35-turbo",
    #     "api_key": AOAIKey,
    #     "api_type": "azure",
    #     "api_base": AOAIEndpoint,
    #     "api_version": "2023-07-01-preview",
    # },
    # {
    #     "model": "gpt-35-turbo-16k",
    #     "api_key": AOAIKey,
    #     "api_type": "azure",
    #     "api_base": AOAIEndpoint,
    #     "api_version": "2023-07-01-preview",
    # },
    {
        "model": "gpt-4",
        "api_key": AOAIKey,
        "api_type": "azure",
        "api_base": AOAIEndpoint,
        "api_version": "2023-07-01-preview",
    },
    {
        "model": "gpt-4-32k",
        "api_key": AOAIKey,
        "api_base": AOAIEndpoint,
        "api_type": "azure",
        "api_version": "2023-07-01-preview"
    },
]

## エージェントの定義

今回の例では、以下の４つのエージェントを定義し、1週間分の夕食の献立を相談しながら決めてもらう。

- floor_manager
ユーザーとの対話を担当
- chef
メニューを考案する
- doctor
医学的立場からメニューを修正する
- kitchen_manager
メニューに必要な食材などを調達する


「system_message=」でエージェントの役割を定義する。

今回はコードの生成と実行はしないが、必要な場合は
「code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},」
を利用する。

「groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)」
で、参加させるエージェントと実行ラウンド数を決める。

人間の介入を受け付ける場合は、以下を変更 「human_input_mode="TERMINATE"」→「human_input_mode="ALWAYS"」

In [20]:
llm_config = {"config_list": config_list}
floor_manager = autogen.UserProxyAgent(
   name="floor_manager",
   system_message="人間との対話を通じて、ChefやDoctor、Kitchen_Managerと相談しながら調理や食材に関する課題を解決してください。",
#   code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
   human_input_mode="ALWAYS"
)
chef = autogen.AssistantAgent(
    name="chef",
    system_message="世界中の料理を知り尽くした料理人です。健康面はdoctorが検討するので、考慮する必要はありません。味についてのみ検討したレシピを考案し、最高の料理を提供します。考案したメニューをdoctorと相談して、健康面を考慮しながら修正してください。メニューが決定したらkitchen_managerに食材リストの提出を依頼してください。",
    llm_config=llm_config,
)
doctor = autogen.AssistantAgent(
    name="doctor",
    system_message="chefが提案したメニューを医学的な立場で検証し、Chefに修正を依頼してください。",
    llm_config=llm_config,
)
kitchen_manager = autogen.AssistantAgent(
    name="kitchen_manager",
    system_message="料理にかかる費用や必要な食材や調味料を管理します。chefが考案しメニューから必要な食材を検討し、その調達を指示してください。",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[floor_manager, chef, doctor, kitchen_manager], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## チャット開始

議論してもらうテーマを与えると議論が開始される。

実行時に「chef(to chat_manager):」のように誰宛ての会話なのかが表示される点に注目。

In [21]:
floor_manager.initiate_chat(
    manager, message=
    "私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。"
)
# type exit to terminate the chat

floor_manager (to chat_manager):

私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。

--------------------------------------------------------------------------------
私は高血圧の持病を持っていますが、食に熱心で、常においしいものを食べたいと考えています。このことを考慮して、1週間分の夕食の献立を考案し、それに必要な食材を調達するためのリストを生成してください。

--------------------------------------------------------------------------------
chef (to chat_manager):

承知しました。高血圧は熱を使用する調理法や塩分の摂取量に注意を払う必要があります。以下がその日の夕食のメニューとなります。

1. フレッシュサーモンのグリル野菜添え
2. イタリアンハーブチキン、ブラウンライス、スチームブロッコリー
3. 白身魚フィレのレモンハーブ蒸し
4. 鶏胸肉のピリ辛パプリカソース
5. タイ風雑穀サラダ
6. レンズ豆と野菜の煮物、クスクス添え
7. シーフードトマトソースパスタ （全粒穀物）

これらのレシピは低塩分で、火を通し過ぎない調理法で健康を考慮しています。いかがでしょうか？

--------------------------------------------------------------------------------
doctor (to chat_manager):

先程のメニューはとても美味しそうですね。しかし、高血圧を持つ方にとっては若干改善が必要になります。以下にレビューと修正点を並べてみます。

1. フレッシュサーモンのグリル野菜添え: お魚は非常に体に良いですが、ソース等に注意するようにしてください。バターやこってりしたソースは避け、薄味であることを確認しましょう。

2. イタリアンハーブチキン、ブラウンライス、スチームブロッコリー: このメニューは適切です。

3